In [7]:
import psycopg2
import pandas as pd
import os
from dotenv import load_dotenv

load_dotenv()

DATABASE_URL = os.getenv("DATABASE_URL")

if not DATABASE_URL:
    raise RuntimeError("❌ DATABASE_URL is not set in your environment!")

print("DATABASE_URL loaded!")

def get_pg_connection():
    try:
        conn = psycopg2.connect(DATABASE_URL)
        print("✅ Connected to Postgres database")
        return conn
    except Exception as e:
        print("❌ Connection Error:", e)
        raise e

conn = get_pg_connection()


DATABASE_URL loaded!
✅ Connected to Postgres database


In [8]:
def run_sql(sql):
    try:
        df = pd.read_sql_query(sql, conn)
        print(f"✔ Query OK — {len(df)} rows")
        return df
    except Exception as e:
        print("❌ SQL Error:", e)


In [9]:
sql_test_1 = """
SELECT 
    r.regionid,
    r.region,
    c.countryid,
    c.country
FROM country c
JOIN region r ON c.regionid = r.regionid
ORDER BY r.region, c.country
LIMIT 20;
"""

df1 = run_sql(sql_test_1)
df1


C:\Users\91889\AppData\Local\Temp\ipykernel_3048\1635914683.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql, conn)


✔ Query OK — 20 rows


,regionid,region,countryid,country
0,1,British Isles,10,Ireland
1,1,British Isles,19,UK
2,2,Central America,12,Mexico
3,3,Eastern Europe,14,Poland
4,4,North America,5,Canada
5,4,North America,20,USA
6,5,Northern Europe,6,Denmark
7,5,Northern Europe,17,Sweden
8,6,Scandinavia,7,Finland
9,6,Scandinavia,13,Norway


In [10]:
sql_test_2 = """
SELECT 
    c.customerid,
    c.firstname || ' ' || c.lastname AS customer_name,
    c.city,
    co.country,
    r.region
FROM customer c
JOIN country co ON c.countryid = co.countryid
JOIN region r ON co.regionid = r.regionid
ORDER BY c.customerid
LIMIT 20;
"""

df2 = run_sql(sql_test_2)
df2


✔ Query OK — 20 rows


C:\Users\91889\AppData\Local\Temp\ipykernel_3048\1635914683.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql, conn)


,customerid,customer_name,city,country,region
0,1,Alejandra Camino,Madrid,Spain,Southern Europe
1,2,Alexander Feuer,Leipzig,Germany,Western Europe
2,3,Ana Trujillo,Mexico D.F.,Mexico,Central America
3,4,Anabela Domingues,Sao Paulo,Brazil,South America
4,5,Andre Fonseca,Campinas,Brazil,South America
5,6,Ann Devon,London,UK,British Isles
6,7,Annette Roulet,Toulouse,France,Western Europe
7,8,Antonio Moreno,Mexico D.F.,Mexico,Central America
8,9,Aria Cruz,Sao Paulo,Brazil,South America
9,10,Art Braunschweiger,Lander,USA,North America


In [13]:
sql_test_3 = """
SELECT 
    p.productid,
    p.productname,
    p.productunitprice,
    pc.productcategory,
    pc.productcategorydescription
FROM product p
JOIN productcategory pc ON p.productcategoryid = pc.productcategoryid
ORDER BY p.productid
LIMIT 20;
"""

df3 = run_sql(sql_test_3)
df3


✔ Query OK — 20 rows


C:\Users\91889\AppData\Local\Temp\ipykernel_3048\1635914683.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql, conn)


,productid,productname,productunitprice,productcategory,productcategorydescription
0,1,Alice Mutton,39.00,Meat/Poultry,Prepared meats
1,2,Aniseed Syrup,10.00,Condiments,"Sweet and savory sauces, relishes, spreads, an..."
2,3,Boston Crab Meat,18.40,Seafood,Seaweed and fish
3,4,Camembert Pierrot,34.00,Dairy Products,Cheeses
4,5,Carnarvon Tigers,62.50,Seafood,Seaweed and fish
5,6,Chai,18.00,Beverages,"Soft drinks, coffees, teas, beers, and ales"
6,7,Chang,19.00,Beverages,"Soft drinks, coffees, teas, beers, and ales"
7,8,Chartreuse verte,18.00,Beverages,"Soft drinks, coffees, teas, beers, and ales"
8,9,Chef Anton's Cajun Seasoning,22.00,Condiments,"Sweet and savory sauces, relishes, spreads, an..."
9,10,Chef Anton's Gumbo Mix,21.35,Condiments,"Sweet and savory sauces, relishes, spreads, an..."


In [11]:
sql_test_4 = """
SELECT 
    o.orderid,
    c.firstname || ' ' || c.lastname AS customer_name,
    p.productname,
    o.orderdate,
    o.quantityordered
FROM orderdetail o
JOIN customer c ON o.customerid = c.customerid
JOIN product p ON o.productid = p.productid
ORDER BY o.orderid
LIMIT 20;
"""

df4 = run_sql(sql_test_4)
df4


✔ Query OK — 20 rows


C:\Users\91889\AppData\Local\Temp\ipykernel_3048\1635914683.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql, conn)


,orderid,customer_name,productname,orderdate,quantityordered
0,1,Alejandra Camino,Teatime Chocolate Biscuits,2012-08-14,1
1,2,Alejandra Camino,Guarana Fantastica,2012-08-14,6
2,3,Alejandra Camino,Steeleye Stout,2012-08-14,4
3,4,Alejandra Camino,Nord-Ost Matjeshering,2012-08-15,6
4,5,Alejandra Camino,Ravioli Angelo,2012-08-15,2
5,6,Alejandra Camino,Nord-Ost Matjeshering,2012-09-16,10
6,7,Alejandra Camino,Perth Pasties,2012-09-16,10
7,8,Alejandra Camino,Tourtiere,2012-09-16,5
8,9,Alejandra Camino,Nord-Ost Matjeshering,2014-03-02,1
9,10,Alejandra Camino,Camembert Pierrot,2014-03-02,10


In [14]:
def check_empty(df, label):
    if df is None or len(df) == 0:
        print(f"❌ FAIL: {label} returned no rows")
    else:
        print(f"✅ PASS: {label} looks good ({len(df)} rows)")

check_empty(df1, "Region → Country Mapping")
check_empty(df2, "Customer table")
check_empty(df3, "Product + ProductCategory tables")
check_empty(df4, "OrderDetail full join")


✅ PASS: Region → Country Mapping looks good (20 rows)
✅ PASS: Customer table looks good (20 rows)
✅ PASS: Product + ProductCategory tables looks good (20 rows)
✅ PASS: OrderDetail full join looks good (20 rows)
